In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
DOMAIN = 'semparse_cdr' # test, spouse, semparse_cdr

In [3]:
import sys
import os

os.environ['SNORKELDB'] = 'postgres:///{}'.format(DOMAIN)

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'semparse_cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [4]:
candidates = session.query(candidate_class).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 13780


In [5]:
%%time

from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'semparse_cdr':
    import bz2
    import cPickle
    if 'ctd_unspecified' not in globals():
        with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
            ctd_unspecified, ctd_therapy, ctd_marker = cPickle.load(ctd_f)
    user_lists = {
        'uncertain': ['combin', 'possible', 'unlikely'],
        'causal': ['causes', 'caused', 'induce', 'induces', 'induced', 'associated with'],
        'treat': ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap'],
        'procedure': ['inject', 'administrat'],
        'patient': ['in a patient with', 'in patients with'],
        'weak': ['none', 'although', 'was carried out', 'was conducted', 'seems', 
                 'suggests', 'risk', 'implicated', 'the aim', 'to investigate',
                 'to assess', 'to study'],
        'ctd_unspecified': ctd_unspecified,
        'ctd_therapy': ctd_therapy,
        'ctd_marker': ctd_marker,
    }

CPU times: user 27.3 s, sys: 617 ms, total: 27.9 s
Wall time: 29.5 s


In [6]:
from snorkel.semantic import (get_left_phrases, get_right_phrases, 
                              get_between_phrases, get_sentence_phrases)
c = candidates[1]
print c[0].get_parent()._asdict()['text']
print c

TDP is a side-effect that has led to withdrawal of several drugs from the market (e.g. terfenadine and terodiline). 
ChemicalDisease(Span("terodiline)", sentence=6413, chars=[103,113], words=[19,20]), Span("TDP", sentence=6413, chars=[0,2], words=[0,0]))


In [7]:
# print c[0].get_attrib_tokens(a='words')
# c[0].get_attrib_tokens(a='entity_cids')

In [8]:
# for p in get_left_phrases(c[0], cmp='.leq', num=3, n_max=3):
#     print getattr(p, 'text')

In [9]:
# c[0].get_parent()._asdict()

In [10]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [11]:
# c = sv.get_selected()
# print c
# print c[0].get_parent()._asdict()['pos_tags']
# print c[0].get_parent()._asdict()['ner_tags']

In [12]:
# hash(c)

In [13]:
# examples[0].display()

In [14]:
# examples[28].candidate[0].get_parent()

In [15]:
from snorkel.semantic import SemanticParser

%time sp = SemanticParser(candidate_class, user_lists, absorb=False)

Created grammar with 272 rules
CPU times: user 2.99 ms, sys: 11 ms, total: 14 ms
Wall time: 15.4 ms


In [25]:
%time results = sp.evaluate(examples,\
                            show_everything=False,\
                            show_explanation=False,\
                            show_candidate=False,\
                            show_sentence=False,\
                            show_parse=False,\
                            show_passing=True,\
                            only=[])
results

P: ('.root', ('.label', ('.bool', True), ('.call', ('.in', ('.extract_text', ('.right', ('.arg', ('.int', 1)), ('.string', '.eq'), ('.int', 1), ('.string', 'words')))), ('.arg_to_string', ('.arg', ('.int', 2))))))
P: ('.root', ('.label', ('.bool', True), ('.and', ('.call', ('.in', ('.extract_text', ('.right', ('.arg', ('.int', 1)), ('.string', '.eq'), ('.int', 1), ('.string', 'words')))), ('.arg_to_string', ('.arg', ('.int', 2)))), ('.any', ('.map', ('.in', ('.arg_to_string', ('.arg', ('.int', 1)))), ('.list', ('.string', u'induc'), ('.string', u'assoc')))))))
P: ('.root', ('.label', ('.bool', False), ('.and', ('.any', ('.map', ('.composite_or', ('.contains',), ('.user_list', ('.string', u'treat'))), ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2))))))), ('.call', ('.in', ('.extract_text', ('.left', ('.arg', ('.int', 2)), ('.string', '.leq'), ('.int', 100), ('.string', 'chars')))), ('.arg_to_string', ('.arg', ('.int', 1)))))))
P: ('.root', ('.label'

,Correct,Passing,Failing,Redundant,Error,Unknown
LF_c_cause_d,1,0,0,0,0,0
LF_c_d,0,1,0,0,0,0
LF_c_induced_d,0,1,0,0,0,0
LF_c_treat_d,0,1,0,0,0,0
LF_c_treat_d_wide,0,1,0,0,0,0
LF_ctd_marker_c_d,0,1,1,0,0,0
LF_ctd_marker_induce,0,5,5,0,0,0
LF_ctd_therapy_treat,0,2,2,0,0,0
LF_ctd_unspecified_treat,0,2,2,0,0,0
LF_ctd_unspecified_induce,0,7,3,0,0,0


In [17]:
import numpy as np
print np.sum(results, axis=0)

Correct       1
Passing      43
Failing      16
Redundant     0
Error         0
Unknown       0
dtype: int64


In [18]:
(correct, passing, failing, redundant, erroring, unknown) = sp.LFs
LFs = correct + passing
print len(LFs)

44


In [19]:
from snorkel.semantic import sem_to_str

sem = ('.root', ('.label', ('.bool', True), ('.and', ('.any', ('.map', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.user_list', ('.string', u'causal')))), ('.not', ('.call', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.string', u'not'))))))
print sem
print sem_to_str(sem)

('.root', ('.label', ('.bool', True), ('.and', ('.any', ('.map', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.user_list', ('.string', u'causal')))), ('.not', ('.call', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.string', u'not'))))))
return -1 if (any(map(in text(between([arg1,arg2])), 'CAUSAL')) and not (call(in text(between([arg1,arg2])), 'not'))) else 0


In [20]:
# sp.grammar.print_chart(nested=True)

In [21]:
# sp.grammar.print_grammar()